# Spark SQL and DataFrames

## Configuring `JAVA_HOME`

**Optional configuration (in case you need a different JVM version)**

In [2]:
import os

# En nuestro ordenador personal, si no esta definida la variable JAVA_HOME, deberemos indicarla
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

# En los laboratorios docentes, sera necesario utilizar la siguiente
# os.environ["JAVA_HOME"] = "/usr/"

os.environ["JAVA_HOME"]

'/home/maes/.sdkman/candidates/java/current'

**Import libraries, check versions**

In [3]:
import pyspark
print(pyspark.__version__)

3.5.3


In [4]:
%%bash
java -version

openjdk version "21.0.4" 2024-07-16 LTS
OpenJDK Runtime Environment Temurin-21.0.4+7 (build 21.0.4+7-LTS)
OpenJDK 64-Bit Server VM Temurin-21.0.4+7 (build 21.0.4+7-LTS, mixed mode, sharing)


## Opening SparkSession

In [5]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .master("local[*]")
         #.config("spark.driver.cores", 1)
         .appName("101 Spark DataFrames")
         .getOrCreate() )

sc = spark.sparkContext
spark

24/11/18 00:37:14 WARN Utils: Your hostname, maes-GE72-7RE resolves to a loopback address: 127.0.1.1; using 192.168.1.58 instead (on interface enp3s0)
24/11/18 00:37:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/18 00:37:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/18 00:37:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/18 00:37:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# RDD vs Data Frames

## Use low level RDD API

In [14]:
# In Python
# Create an RDD of tuples (name, age)
dataRDD = sc.parallelize([("Brooke", 20), ("Denny", 31), ("Jules", 30),
("TD", 35), ("Brooke", 25)])
# Use map and reduceByKey transformations with their lambda
# expressions to aggregate and then compute average
agesRDD = (dataRDD
.map(lambda x: (x[0], (x[1], 1)))
.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
.map(lambda x: (x[0], x[1][0]/x[1][1])))

In [15]:
agesRDD.collect()

[('Brooke', 22.5), ('Denny', 31.0), ('TD', 35.0), ('Jules', 30.0)]

## Use high-level Dataframe API

In [16]:
from pyspark.sql.functions import avg

data_df = spark.createDataFrame([("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)], ["name", "age"])
# Group the same names together, aggregate their ages, and compute an average
avg_df = data_df.groupBy("name").agg(avg("age"))
# Show the results of the final execution
avg_df.show()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Denny|    31.0|
| Jules|    30.0|
|    TD|    35.0|
+------+--------+

